In [1]:
import pandas as pd

## DATA SETS ---------
smsData = pd.read_csv("spam.csv", encoding = "latin-1", usecols=["v1", "v2"])## importa o data set como um data-frame, usa o latin-1 com decoding para ler o ficheiro e seleciona as colunas v1 e v2 como as unicas a ser usadas (o ficheiro tem mais 3 colunas mas vazias) 

## TRANSFORMS
smsData.columns = ["Status", "Mensagem"] #renomeia os hearders das colunas

print("Linhas x Colunas", smsData.shape) #print do tamanho da matriz
smsData.head()  # print das primeiras 5 linhas da mat

(5572, 2)


,Status,Mensagem
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [70]:
print("Distribuição")
print(smsData['Status'].value_counts()) #Quantidade da ham x spam na tabela
print("")
print("Frequência Relativa")
print(smsData['Status'].value_counts(normalize=True)) #Frequencia relativa

Distribuição
ham     4825
spam     747
Name: Status, dtype: int64

Frequência Relativa
ham     0.865937
spam    0.134063
Name: Status, dtype: float64


In [74]:
#Divisão em Sets
data_randomized = smsData.sample(frac=1, random_state=666) # Faz suffle da tabela - frac é a fração dos dados a usar (0,0 a 1,0) - random-state = (int) é a seed que o sistema vai usar como "fator de entropia" para gerar o "random"

# Definir o percentagem de dados a usar (0 a 1)
training_test_index = round(len(data_randomized) * 0.8)

# Criação do grupo de teste e treino
training_set = data_randomized[:training_test_index].reset_index(drop=True) #Faz splice dos dados com base no <inserir nome> que serve como indicador da percentagem de dados a usar. neste caso os primeiros 80% da tabela - reset_index faz reset ao indice da tabela
test_set = data_randomized[training_test_index:].reset_index(drop=True) #Faz splice dos dados com base no <inserir nome> que serve como indicador da percentagem de dados a usar. neste caso os dados que sobram após "ler" 80% da tabela - reset_index faz reset ao indice da tabela


print("Dados de Treino")
print(training_set['Status'].value_counts())
print("")
print(training_set['Status'].value_counts(normalize = True))

print("")

print("Dados de Teste")
print(test_set['Status'].value_counts())
print("")
print(test_set['Status'].value_counts(normalize = True))

Dados de Treino
ham     3852
spam     606
Name: Status, dtype: int64

ham     0.864065
spam    0.135935
Name: Status, dtype: float64

Dados de Teste
ham     973
spam    141
Name: Status, dtype: int64

ham     0.873429
spam    0.126571
Name: Status, dtype: float64


In [72]:
#Limpeza dos dados
training_set['Mensagem'] = training_set['Mensagem'].str.replace('\W', ' ') # Limpa a pontuação
training_set['Mensagem'] = training_set['Mensagem'].str.lower()  #Transforma tudo em letra pequena
training_set.head()

,Status,Mensagem
0,ham,convey my regards to him
1,ham,û_ anyway many good evenings to u s
2,ham,my sort code is and acc no is the bank is n...
3,ham,sorry i din lock my keypad
4,spam,hi babe its chloe how r u i was smashed on s...


In [66]:
training_set['Mensagem'] = training_set['Mensagem'].str.split()

vocabulary = []
for sms in training_set['Mensagem']:
   for word in sms:
      vocabulary.append(word)

vocabulary = list(set(vocabulary))

In [67]:
len(vocabulary)

7766

In [58]:
word_counts_per_sms = {'secret': [2,1,1],
                       'prize': [2,0,1],
                       'claim': [1,0,1],
                       'now': [1,0,1],
                       'coming': [0,1,0],
                       'to': [0,1,0],
                       'my': [0,1,0],
                       'party': [0,1,0],
                       'winner': [0,0,1]
                      }

word_counts = pd.DataFrame(word_counts_per_sms)
word_counts.head()

,secret,prize,claim,now,coming,to,my,party,winner
0,2,2,1,1,0,0,0,0,0
1,1,0,0,0,1,1,1,1,0
2,1,1,1,1,0,0,0,0,1


In [39]:
word_counts_per_sms = {unique_word: [0] * len(training_set['Mensagem']) for unique_word in vocabulary}

for index, sms in enumerate(training_set['Mensagem']):
   for word in sms:
      word_counts_per_sms[word][index] += 1

In [41]:
word_counts = pd.DataFrame(word_counts_per_sms)
word_counts.head()

,ringing,f4q,thot,spiral,advise,zyada,ì¼1,antha,carry,slide,...,cuddle,message,mre,waiting,alone,opposed,seen,sitll,ay,united
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
training_set_clean = pd.concat([training_set, word_counts], axis=1)
training_set_clean.head()

,Status,Mensagem,ringing,f4q,thot,spiral,advise,zyada,ì¼1,antha,...,cuddle,message,mre,waiting,alone,opposed,seen,sitll,ay,united
0,ham,"[convey, my, regards, to, him]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[û_, anyway, many, good, evenings, to, u, s]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[my, sort, code, is, and, acc, no, is, the, ba...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,"[sorry, i, din, lock, my, keypad]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,spam,"[hi, babe, its, chloe, how, r, u, i, was, smas...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [44]:
# Isolating spam and ham messages first
spam_messages = training_set_clean[training_set_clean['Status'] == 'spam']
ham_messages = training_set_clean[training_set_clean['Status'] == 'ham']

# P(Spam) and P(Ham)
p_spam = len(spam_messages) / len(training_set_clean)
p_ham = len(ham_messages) / len(training_set_clean)

# N_Spam
n_words_per_spam_message = spam_messages['Mensagem'].apply(len)
n_spam = n_words_per_spam_message.sum()

# N_Ham
n_words_per_ham_message = ham_messages['Mensagem'].apply(len)
n_ham = n_words_per_ham_message.sum()

# N_Vocabulary
n_vocabulary = len(vocabulary)

# Laplace smoothing
alpha = 1

In [45]:
# Initiate parameters
parameters_spam = {unique_word:0 for unique_word in vocabulary}
parameters_ham = {unique_word:0 for unique_word in vocabulary}

# Calculate parameters
for word in vocabulary:
   n_word_given_spam = spam_messages[word].sum() # spam_messages already defined
   p_word_given_spam = (n_word_given_spam + alpha) / (n_spam + alpha*n_vocabulary)
   parameters_spam[word] = p_word_given_spam

   n_word_given_ham = ham_messages[word].sum() # ham_messages already defined
   p_word_given_ham = (n_word_given_ham + alpha) / (n_ham + alpha*n_vocabulary)
   parameters_ham[word] = p_word_given_ham

In [46]:
import re

def classify(message):
   '''
   message: a string
   '''

   message = re.sub('\W', ' ', message)
   message = message.lower().split()

   p_spam_given_message = p_spam
   p_ham_given_message = p_ham

   for word in message:
      if word in parameters_spam:
         p_spam_given_message *= parameters_spam[word]

      if word in parameters_ham: 
         p_ham_given_message *= parameters_ham[word]

   print('P(Spam|message):', p_spam_given_message)
   print('P(Ham|message):', p_ham_given_message)

   if p_ham_given_message > p_spam_given_message:
      print('Label: Ham')
   elif p_ham_given_message < p_spam_given_message:
      print('Label: Spam')
   else:
      print('Equal proabilities, have a human classify this!')

In [47]:
classify('WINNER!! This is the secret code to unlock the money: C3421.')

P(Spam|message): 7.551469020619691e-26
P(Ham|message): 3.13163390665657e-27
Label: Spam


In [48]:
classify("Sounds good, Tom, then see u there")

P(Spam|message): 4.914731817244035e-25
P(Ham|message): 4.052730830087535e-21
Label: Ham


In [49]:
def classify_test_set(message):
   '''
   message: a string
   '''

   message = re.sub('\W', ' ', message)
   message = message.lower().split()

   p_spam_given_message = p_spam
   p_ham_given_message = p_ham

   for word in message:
      if word in parameters_spam:
         p_spam_given_message *= parameters_spam[word]

      if word in parameters_ham:
         p_ham_given_message *= parameters_ham[word]

   if p_ham_given_message > p_spam_given_message:
      return 'ham'
   elif p_spam_given_message > p_ham_given_message:
      return 'spam'
   else:
      return 'needs human classification'

In [50]:
test_set['predicted'] = test_set['Mensagem'].apply(classify_test_set)
test_set.head()

,Status,Mensagem,predicted
0,ham,S...from the training manual it show there is ...,ham
1,spam,Do you want a new Video phone? 600 anytime any...,spam
2,ham,True. Its easier with her here.,ham
3,ham,Midnight at the earliest,ham
4,ham,"All done, all handed in. Don't know if mega sh...",ham


In [51]:
correct = 0
total = test_set.shape[0]

for row in test_set.iterrows():
   row = row[1]
   if row['Status'] == row['predicted']:
      correct += 1

print('Correct:', correct)
print('Incorrect:', total - correct)
print('Accuracy:', correct/total)

Correct: 1107
Incorrect: 7
Accuracy: 0.9937163375224417


In [52]:
classify("Chupamos valadão, ahhhhhhh caralho odeio esta cadeira")

P(Spam|message): 0.13593539703903096
P(Ham|message): 0.8640646029609691
Label: Ham
